**1. Configuração do Virtual Environment**

1.Criar o Virtual Environment: Execute os comandos abaixo no terminal na pasta do projeto:

*python -m venv venv*

2.Ativar o Virtual Environment:

Windows: *venv\Scripts\activate*

Linux/Mac: *source venv/bin/activate*

3.Instalar as Bibliotecas Necessárias:

In [3]:
pip install pandas requests plyer

Note: you may need to restart the kernel to use updated packages.


4.Salvar as Dependências no *requirements.txt*:

In [6]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


**2. Pipeline de Dados Completo com Sistema de Alerta**

O código final do pipeline já incorpora o sistema de alerta usando o módulo plyer:

In [24]:
import requests
import pandas as pd
import sqlite3
from plyer import notification

# Função de alerta
def alerta(titulo, mensagem):
    notification.notify(
        title=titulo,
        message=mensagem,
        app_name="Data Pipeline",
        timeout=10
    )

# 1. Extração de Dados
def extrair_dados():
    url = "https://restcountries.com/v3.1/all"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Criar DataFrame com colunas relevantes
            countries = []
            for country in data:
                name = country.get("name", {}).get("common", "Unknown")
                region = country.get("region", "Unknown")
                currencies = "; ".join(country.get("currencies", {}).keys())
                languages = "; ".join(country.get("languages", {}).values()) if "languages" in country else "Unknown"
                countries.append({
                    "country": name,
                    "continent": region,
                    "currencies": currencies,
                    "languages": languages
                })
            return pd.DataFrame(countries)
        else:
            alerta("Erro na API", f"Código de status {response.status_code}")
            print(f"Erro na API: Código {response.status_code}")
            return None
    except Exception as e:
        alerta("Erro de Conexão", str(e))
        print(f"Erro de conexão: {str(e)}")
        return None

# 2. Tratamento de Dados
def tratar_dados(df):
    # Tabela Continentes
    df_continentes = df[["country", "continent"]].copy()
    df_continentes["continent"] = df_continentes["continent"].fillna("Unknown")

    # Tabela Moedas
    if "currencies" in df.columns:
        df_moedas = df[["country", "currencies"]].rename(columns={"currencies": "currency"}).dropna()
    else:
        alerta("Erro no Tratamento", "A coluna 'currencies' não está presente nos dados.")
        df_moedas = pd.DataFrame(columns=["country", "currency"])

    # Tabela Idiomas
    if "languages" in df.columns:
        df_idiomas = df[["country", "languages"]].rename(columns={"languages": "language"}).dropna()
    else:
        alerta("Erro no Tratamento", "A coluna 'languages' não está presente nos dados.")
        df_idiomas = pd.DataFrame(columns=["country", "language"])

    return df_continentes, df_moedas, df_idiomas

# 3. Carga no Banco de Dados
def carregar_banco(df_continentes, df_moedas, df_idiomas, db_name="dados_tratados.db"):
    conn = sqlite3.connect(db_name)
    try:
        df_continentes.to_sql("continentes", conn, if_exists="replace", index=False)
        df_moedas.to_sql("moedas", conn, if_exists="replace", index=False)
        df_idiomas.to_sql("idiomas", conn, if_exists="replace", index=False)
        alerta("Carga Completa", f"Dados salvos no banco {db_name}.")
        print("Dados carregados no banco com sucesso.")
    except Exception as e:
        alerta("Erro na Carga", f"Erro ao salvar os dados no banco: {str(e)}")
        print(f"Erro ao salvar os dados no banco: {str(e)}")
    finally:
        conn.close()

# 4. Validação
def validar_dados(db_name="dados_tratados.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    try:
        for tabela in ["continentes", "moedas", "idiomas"]:
            cursor.execute(f"SELECT COUNT(*) FROM {tabela}")
            resultado = cursor.fetchone()[0]
            print(f"Tabela {tabela} contém {resultado} registros.")
    except Exception as e:
        print(f"Erro na validação dos dados: {str(e)}")
    finally:
        conn.close()

# Execução do Pipeline
if __name__ == "__main__":
    try:
        # Extração
        df_raw = extrair_dados()
        if df_raw is not None:
            # Tratamento
            df_continentes, df_moedas, df_idiomas = tratar_dados(df_raw)

            # Carga
            carregar_banco(df_continentes, df_moedas, df_idiomas)

            # Validação
            validar_dados()
        else:
            alerta("Erro no Pipeline", "Nenhum dado foi extraído.")
    except Exception as e:
        alerta("Erro no Pipeline", str(e))
        print("Erro no Pipeline", str(e))


Dados carregados no banco com sucesso.
Tabela continentes contém 250 registros.
Tabela moedas contém 250 registros.
Tabela idiomas contém 250 registros.
